<div style="font-family: 'TeX Gyre Termes', serif; color: black; font-size: 18px;">

### **Pixel Based Full-Waveform LiDAR Simulation Setting in DIRSIG.**  
**Author: Ramesh Bhatta, CIS, RIT**  
 
*This code file helps calculating required parameters for setting full-waveform lidar simulation in DIRSIG (https://dirsig.cis.rit.edu). The program is set-up to collect the line of single nadir viewing pixel to collect waveform signal.*

</div>

In [1]:
## Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import math
from tabulate import tabulate

In [2]:
## Set flight and scene coverage details

# Flight Details
altitude = 5000  # Flight altitude (m)
speed = 100   # Flight speed (m/s)
det_pixel_size = 50 # Detector pixel size in microns (um)
gnd_pixel_size = 5 # Ground pixel size (m)

# Scene coverage
scene_start = [100, 0] # Coordinate for scene start
scene_end = [100, 500] # Coordinate for scene end

across_len = scene_end[0]- scene_start[0] if scene_end[0]- scene_start[0]!=0 else gnd_pixel_size    # Across-track coverage (m) --->
                                                                                                    # currently I am scanning a line of pixels so
                                                                                                    # it will be same as gnd_pixel_size
along_len = scene_end[1]- scene_start[1] if scene_end[1]- scene_start[1]!=0 else gnd_pixel_size   # Along-track coverage (m)


In [3]:
## Footprint Analysis based on the pixel size

def footprint_detail(pixel_size):

    f = (np.pi / 2) * pixel_size**2
    d = pixel_size * np.sqrt(2)

    print(f'The footprint area based on pixel size of {pixel_size} m is {round(f, 3)} m2')
    print(f'And the footprint size based on pixel size of {pixel_size} m is {round(d, 3)} m')

    return None

# Footprint Info
footprint_detail(gnd_pixel_size)

The footprint area based on pixel size of 5 m is 39.27 m2
And the footprint size based on pixel size of 5 m is 7.071 m


In [4]:
## Collection Details
"""
Sim outlook based on above details
Assuming that flight is along the Y-axis of scene (North up along Y)
and we are illuminating the center of pixel looking directly below from the ALS
"""

collect_start_coordinate = [scene_start[0] - gnd_pixel_size/2, scene_start[1] + gnd_pixel_size / 2]

# Total time to move across the scene would be:
total_time = along_len/speed

# Number of lines to be scanned in total time would be:
num_lines = along_len/gnd_pixel_size

# So the time for one-individual line collection would be:
line_instance_time = total_time/num_lines

# Now the collection would look like:
data = []
for i in range(int(num_lines)):
    instantaneous_time = (i + 1) * line_instance_time
    instantaneous_location = [collect_start_coordinate[0], collect_start_coordinate[1] + (i) * gnd_pixel_size]
    data.append([i + 1, instantaneous_time-line_instance_time, instantaneous_location]) ## Assuming collection starts from time 0.00 not from instananeous time
    if i == 0:
        print(f'Scan start ground location: {instantaneous_location}')
    elif i == int(num_lines)-1:
        print(f'Scan End ground location: {instantaneous_location}')
    else:
        continue

# Ignore this (I used this to match waveform with LAI pixels)
print(f'Corresponding X-pixel location for LAI mapping in scene: {int(scene_start[0]/gnd_pixel_size)}th pixel') # Considering a transect collection across the scene

# Print sample of collection locations in a table using tabulate
headers = ["Pixel Count", "Time Instance", "Ground Projected Platform Location"]
print('\nNeed to make sure our collection looks like this:')
print(tabulate(data[:10], headers=headers, tablefmt="fancy_grid", colalign=("center", "center", "center")))

Scan start ground location: [97.5, 2.5]
Scan End ground location: [97.5, 497.5]
Corresponding X-pixel location for LAI mapping in scene: 20th pixel

Need to make sure our collection looks like this:
╒═══════════════╤═════════════════╤══════════════════════════════════════╕
│  Pixel Count  │  Time Instance  │  Ground Projected Platform Location  │
╞═══════════════╪═════════════════╪══════════════════════════════════════╡
│       1       │        0        │             [97.5, 2.5]              │
├───────────────┼─────────────────┼──────────────────────────────────────┤
│       2       │      0.05       │             [97.5, 7.5]              │
├───────────────┼─────────────────┼──────────────────────────────────────┤
│       3       │       0.1       │             [97.5, 12.5]             │
├───────────────┼─────────────────┼──────────────────────────────────────┤
│       4       │      0.15       │             [97.5, 17.5]             │
├───────────────┼─────────────────┼────────────────

In [5]:
## Function to calculate required parameters to fulfill the flight plan

def calc_sim_params(flight_height, flight_speed, pixel_pitch, gnd_pix_size, across_track_length, along_track_length):

    # Message
    print(" "+"\033[1m" +"\033[4m" + "Simulation Settings:" + "\033[0m")

    # 1. Beam Divergence- Half divergence angle (Gaussian Beam Assumption)

    footprint_diameter = gnd_pix_size * np.sqrt(2)
    half_angle = math.atan2(footprint_diameter, 2 * flight_height)
    print("\n"+"\033[1m" + " Platform and detector:" + "\033[0m")
    print(f' Beam half divergence angle: {np.round(half_angle, 4)} radians')

    # 2. Focal Length of the detector system

    gsd = footprint_diameter
    focal_length = pixel_pitch * 1e-06 * (flight_height / gsd)
    print(f' Focal length of the detector system: {np.round(focal_length, 4)} meters')
    print(f' Approx. Aperture size: {np.round(focal_length / 5, 6)} m')

    # 3. Across track angle
    across_angle = math.atan2(across_track_length, 2 * flight_height)
    print(f' Across-track half angle: {np.round(across_angle, 6)} radians')

    # 4. Timing Synchronization

    num_lines = along_track_length / gnd_pixel_size
    flight_time = along_track_length / flight_speed   # time to cover whole along track path
    one_line_scantime  = flight_time / num_lines      #  time to scan one line
    scan_rate = num_lines / flight_time               # scan rate for line scanner
    print(f' Scan rate: {np.round(scan_rate, 6)} lines/second')

    # 5. Lidar PRR

    num_pixels = across_track_length / gnd_pixel_size   # number of pixels in a line
    time_per_pixel = one_line_scantime / num_pixels
    lidar_PRR = 1 / time_per_pixel
    print(f' LiDAR PRR: {np.round(lidar_PRR, 6)} pulses/second')
    print(f' Number of pixels: {np.int64(np.round(num_pixels))}(across-track) X {np.int64(np.round(num_lines))}(along-track)')
    print(f' Number of waveforms: {np.int64(np.round(num_pixels)) * np.int64(np.round(num_lines))}\n')

    print(" "+"\033[1m" + "Platform motion and capture:" + "\033[0m")
    print(f' Flight time: {np.round(flight_time, 6) - one_line_scantime} seconds') # flight time can be greater, no problem!!
    print(f' Delta-time: {np.round(one_line_scantime, 6)} seconds')
    print(f' Capture-time: {np.round(flight_time, 6) - one_line_scantime} seconds') # Capture time should be this.
    print(f' Range-Gate: Manual')

    return None

 # Calculate the paramters:
calc_sim_params(altitude, speed, det_pixel_size, gnd_pixel_size, across_len, along_len)

 Simulation Settings:

 Platform and detector:
 Beam half divergence angle: 0.0007 radians
 Focal length of the detector system: 0.0354 meters
 Approx. Aperture size: 0.007071 m
 Across-track half angle: 0.0005 radians
 Scan rate: 20.0 lines/second
 LiDAR PRR: 20.0 pulses/second
 Number of pixels: 1(across-track) X 100(along-track)
 Number of waveforms: 100

 Platform motion and capture:
 Flight time: 4.95 seconds
 Delta-time: 0.05 seconds
 Capture-time: 4.95 seconds
 Range-Gate: Manual
